In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv", index_col='Id')
test_df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv", index_col='Id')

In [ ]:
train_df

In [ ]:
sns.histplot(train_df.Pawpularity, bins=20)

In [ ]:
train_df.apply(lambda col:col.unique()) # check out unique values

In [ ]:
train_df.isna().sum() ## no null values

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xg
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR

In [ ]:
X = train_df.iloc[:, :-1]
y = train_df.iloc[:, -1]

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
rf_reg1 = RandomForestRegressor() ## rf regressor
rf_reg1.fit(X_train, y_train)
y_pred_vanilla = rf_reg1.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, y_pred_vanilla)))

In [ ]:
xg_reg1 = xg.XGBRegressor()  #xgb regressor
xg_reg1.fit(X_train, y_train)
y_pred_vanilla = xg_reg1.predict(X_test)
print(np.sqrt(mean_squared_error(y_pred_vanilla, y_test)))

In [ ]:
np.sqrt(mean_squared_error(LinearRegression(normalize=True).fit(X_train, y_train).predict(X_test), y_test))

In [ ]:
np.sqrt(mean_squared_error(LinearSVR(C=0.01, epsilon=0.01, max_iter=50, loss="squared_epsilon_insensitive").fit(X_train, y_train).predict(X_test), y_test))

In [ ]:
## using rf 
params = {
    "n_estimators":[100,300,500],
    "max_depth":[2,4,6],
    "min_samples_split":[2,4,6],
    "oob_score":[True, False]
}

In [ ]:
# grid_rf_reg = GridSearchCV(rf_reg1, params, n_jobs=-1)
# grid_rf_reg.fit(X_train, y_train)

In [ ]:
# grid_rf_reg.best_params_

In [ ]:
# rf_reg2 = RandomForestRegressor(**grid_rf_reg.best_params_).fit(X_train, y_train)
# y_pred_opt = rf_reg2.predict(X_test)
# print(np.sqrt(mean_squared_error(y_test, y_pred_opt)))

Linear model is giving better result than RF in this case, eh - lets see..

In [ ]:
# submission = pd.DataFrame({"Id": test_df.index, "Pawpularity":rf_reg2.predict(test_df)})
submission = pd.DataFrame({"Id": test_df.index, "Pawpularity":LinearRegression(normalize=True).fit(X_train, y_train).predict(test_df)})

In [ ]:
submission.set_index('Id', inplace=True)

In [ ]:
submission.to_csv("submission.csv")